In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("..")  # Go up one level to the UROP directory

In [2]:
import torch
from torch.utils.data import DataLoader
import yaml

import librosa
import matplotlib.pyplot as plt
%matplotlib inline

with open("config.yaml") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

batch_size = config["batch_size"]
lr = config["lr"]
num_epochs = config["num_epochs"]
save_every = config["save_every"]
save_dir = config["save_dir"]
num_workers = config["num_workers"]

In [3]:
from utils import LazyWindowedDataset, train_test_split

# Initialize Dataset
full_train_source_dataset = LazyWindowedDataset(
    root_dir="datasets/BrushlessMotor",
    split="train",
    anomaly_type=['normal'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)

train_source_dataset, val_source_dataset = train_test_split(full_train_source_dataset)

train_loader = DataLoader(train_source_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_loader = DataLoader(val_source_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

In [4]:
test_source_dataset = LazyWindowedDataset(
    root_dir="datasets/BrushlessMotor",
    split="test",
    anomaly_type=['normal', 'anomaly'],
    domain_type=['source', 'target'],
    window_size_ms=100,
    stride_ms=50,
)
test_loader = DataLoader(test_source_dataset, batch_size=batch_size, shuffle=True, num_workers=1, drop_last=True)

In [5]:
from models import BaselineAutoencoder
from utils import train_one_epoch, evaluate, train_model, z_score_normalize, flatten_and_concat

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BaselineAutoencoder()
model.to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [6]:
loss, auc = evaluate(model, test_loader, criterion)
print(f"Initial loss: {loss:.4f}, AUC: {auc:.4f}")

Initial loss: 0.9991, AUC: 0.5246


In [11]:
model, train_losses, val_losses, val_aucs = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,
    save_every=5,
    save_dir=save_dir,
    verbose=False,
)

Epoch [1/20] | Train Loss: 0.790589 | Val Loss: 0.798519 | Val AUC: nan
Time Spent: 182.60s | ETA: 3469.43s | Current Time: 2025-04-22 17:44:11
Epoch [2/20] | Train Loss: 0.788747 | Val Loss: 0.798202 | Val AUC: nan
Time Spent: 181.17s | ETA: 3261.14s | Current Time: 2025-04-22 17:47:13
Epoch [3/20] | Train Loss: 0.786951 | Val Loss: 0.797625 | Val AUC: nan
Time Spent: 180.95s | ETA: 3076.22s | Current Time: 2025-04-22 17:50:14
Epoch [4/20] | Train Loss: 0.784929 | Val Loss: 0.797198 | Val AUC: nan
Time Spent: 181.70s | ETA: 2907.18s | Current Time: 2025-04-22 17:53:15
Epoch [5/20] | Train Loss: 0.783082 | Val Loss: 0.797440 | Val AUC: nan
Time Spent: 181.91s | ETA: 2728.64s | Current Time: 2025-04-22 17:56:17
Checkpoint saved at ./checkpoints/BaselineAutoencoder_epoch_5.pt
Epoch [6/20] | Train Loss: 0.781353 | Val Loss: 0.797890 | Val AUC: nan
Time Spent: 181.71s | ETA: 2543.88s | Current Time: 2025-04-22 17:59:19
Epoch [7/20] | Train Loss: 0.779450 | Val Loss: 0.797738 | Val AUC: nan

KeyboardInterrupt: 

In [12]:
loss, auc = evaluate(model, test_loader, criterion)
print(f"Final loss: {loss:.4f}, AUC: {auc:.4f}")

Final loss: 0.8583, AUC: 0.5919
